MLP modeling
Logistic feature selection
Optuna
Threshold

In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\train_common_v3.3_피처삭제X_군집_1111.csv')
test = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\test_common_v3.3_피처삭제X_군집_1111.csv')
target =  pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\store_train.csv')
submit = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\store_submission (2).csv')

In [5]:
train.shape , target.shape , test.shape , submit.shape

((14940, 1471), (14940, 2), (12225, 1471), (12225, 2))

In [6]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(0, 0)

In [7]:
train_ft = train.iloc[:,1:]
test_ft = test.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

In [8]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [9]:
from sklearn.preprocessing import OneHotEncoder

cols = train_ft.select_dtypes(include=['object']).columns
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(train_ft[cols])

tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft, tmp], axis=1).drop(columns=cols)

tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft, tmp], axis=1).drop(columns=cols)

In [10]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.fit_transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,주구매_중분류_핸드백행사,주구매_중분류_행사,주구매_중분류_행사슈즈,주구매_중분류_행사핸드백,주구매_중분류_향수,주구매_중분류_헤어ACC,주구매_중분류_홈데코,주구매_중분류_화장잡화,주구매_중분류_화장품,주구매_중분류_훼미닌부틱
0,0.041494,0.130682,0.250000,0.750000,0.032258,0.029014,0.050000,0.250000,0.400000,0.300000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.082988,0.090909,0.023810,0.976190,0.006452,0.079304,0.357143,0.166667,0.357143,0.119048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.228216,0.034091,0.210526,0.789474,0.154839,0.174081,0.464912,0.140351,0.175439,0.219298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.377593,0.017045,0.189573,0.810427,0.258065,0.330754,0.379147,0.180095,0.236967,0.203791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.107884,0.062500,0.258065,0.741935,0.103226,0.088975,0.112903,0.612903,0.209677,0.064516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
target = target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_ft, target, test_size=0.2, random_state=42)

In [12]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import optuna
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [15]:
SEED = 42
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Logistic 피처셀렉션
logistic = LogisticRegression(penalty="l2", C=1.0, solver="liblinear", random_state=SEED)
selector = SelectFromModel(logistic)
selected_features = selector.fit_transform(train_ft, target)


selected_cols = selector.get_feature_names_out()
print(f"Selected features: {len(selected_cols)}")

Selected features: 691


In [15]:
# import optuna
# from sklearn.model_selection import cross_val_score, StratifiedKFold
# from sklearn.neural_network import MLPClassifier

# # Optuna 하이퍼파라미터 튜닝
# def objective(trial):
#     # MLPClassifier 하이퍼파라미터 설정
#     n_layers = trial.suggest_int("n_layers", 1, 3)
#     hidden_layer_sizes = tuple([trial.suggest_int(f"n_units_l{i}", 10, 100) for i in range(n_layers)])

#     hp = {
#         "hidden_layer_sizes": hidden_layer_sizes,
#         "activation": trial.suggest_categorical("activation", ["relu", "tanh"]),
#         "solver": trial.suggest_categorical("solver", ["adam", "sgd"]),
#         "alpha": trial.suggest_float("alpha", 1e-5, 1e-1, log=True),
#         "learning_rate_init": trial.suggest_float("learning_rate_init", 1e-5, 1e-1, log=True),
#         "max_iter": trial.suggest_int("max_iter", 10, 200, step=10),
#         "early_stopping": True,
#         "validation_fraction": trial.suggest_float("validation_fraction", 0.1, 0.3, step=0.05),
#         "n_iter_no_change": trial.suggest_int("n_iter_no_change", 5, 15),
#     }


#     model = MLPClassifier(**hp, random_state=SEED)


#     score = cross_val_score(model, selected_features, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()

#     return score


# sampler = optuna.samplers.TPESampler(seed=SEED)
# study = optuna.create_study(direction="maximize", sampler=sampler)
# study.optimize(objective, n_trials=100)


# print("Best hyperparameters:", study.best_params)
# print("Best F1 Score:", study.best_value)


In [16]:
# # Optuna 튜닝 결과
# best_mlp_params = {
#     'hidden_layer_sizes': (88, 89), 
#     'activation': 'relu',
#     'solver': 'adam',
#     'alpha': 0.001230314130334757,
#     'learning_rate_init': 0.0016991055449377132,
#     'max_iter': 500,
#     'validation_fraction': 0.15000000000000002,
#     'n_iter_no_change': 10,
#     'early_stopping': True
# }


# scores = []
# models = []

# for tri, vai in cv.split(train_ft, target):
#     x_train = train_ft.iloc[tri][selected_cols] #logistic 으로 선택된 피처만 사용가능
#     y_train = target.iloc[tri]
#     x_valid = train_ft.iloc[vai][selected_cols]  
#     y_valid = target.iloc[vai]

#     mlp_model = MLPClassifier(**best_mlp_params, random_state=SEED)
#     mlp_model.fit(x_train, y_train)

#     models.append(mlp_model)

#     pred = mlp_model.predict(x_valid)
#     score = f1_score(y_valid, pred, average='macro')
#     scores.append(score)

# #f1 출력
# print("Mean F1 Macro Score:", np.mean(scores))


In [1]:
import copy

In [16]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import numpy as np

# 최적화된 MLP 파라미터
best_mlp_params = {
    'hidden_layer_sizes': (88, 89),
    'activation': 'relu',
    'solver': 'adam',
    'alpha': 0.001230314130334757,
    'learning_rate_init': 0.0016991055449377132,
    'max_iter': 500,
    'validation_fraction': 0.15,
    'n_iter_no_change': 10,
    'early_stopping': True
}

best_xgb_params = {
    'n_estimators': 900,
    'learning_rate': 0.021734683976721573,
    'max_depth': 6,
    'min_child_weight': 5,
    'subsample': 0.7039147285447092,
    'colsample_bytree': 0.5823954097077503,
    'gamma': 2.3107011128410493,
    'random_state': 42
}

best_lgbm_params = {
    'num_leaves': 33,
    'max_depth': 11,
    'learning_rate': 0.01944354088862471,
    'n_estimators': 800,
    'min_child_samples': 65,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 3.6529269182968447e-07,
    'reg_lambda': 3.3190277841665685e-07,
    'random_state': 42
}


# Stacking Base Estimators 설정
estimators = [
    ('mlp', MLPClassifier(**best_mlp_params, random_state=SEED)),
    #('rf', RandomForestClassifier(random_state=SEED)),
    ('xgb', XGBClassifier(random_state=SEED)),
    ('lgbm', LGBMClassifier(random_state=SEED))
]

# Final Estimator 설정
stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

# 교차 검증
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    x_train = train_ft.iloc[tri][selected_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][selected_cols]
    y_valid = target.iloc[vai]

    # Stacking 모델 학습
    stacking_model.fit(x_train, y_train)

    # 학습된 모델을 복사하여 저장
    models.append(copy.deepcopy(stacking_model))

    # 검증 데이터 예측 및 점수 계산
    pred = stacking_model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# 평균 F1 Score 출력
print("Mean F1 Macro Score (Stacking Ensemble):", np.mean(scores))

Mean F1 Macro Score (Stacking Ensemble): 0.727301750399499


In [ ]:
import numpy as np

ensemble_pred = np.mean([model.predict_proba(test_ft[selected_cols]) for model in models], axis=0)
threshold = 0.42
submit["target"] = (ensemble_pred[:, 1] >= threshold).astype(int)
submit.to_csv("mlp_stacking_ensemble_sub.csv", index=False)


In [ ]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

# estimators = [
#     ('mlp', MLPClassifier(**best_mlp_params, random_state=SEED)),
#     ('rf', RandomForestClassifier(random_state=SEED)),
# ]
# stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
# stacking_model.fit(x_train, y_train)

# pred = stacking_model.predict(x_valid)
# score = f1_score(y_valid, pred, average='macro')
# print("Stacking Model F1 Macro Score:", score)


Stacking Model F1 Macro Score: 0.7202856739266279


In [21]:
#임계값 조절 submit
threshold = 0.4
sub_pred = stacking_model.predict_proba(test_ft[selected_cols])
submit["target"] = (sub_pred[:, 1] >= threshold).astype(int)
submit.to_csv("mlp_stacking_sub.csv", index=False)


In [ ]:
# #proba 파일

# mlp_model = MLPClassifier(**best_mlp_params, random_state=SEED)
# mlp_model.fit(train_ft[selected_cols], target)


# pred = mlp_model.predict_proba(test_ft[selected_cols])[:, 1]

# submit["target"] = pred
# submit.to_csv("mlp+optuna+logistic_fs.csv", index=False)

In [ ]:
# #submit 파일

# sub_pred = mlp_model.predict_proba(test_ft[selected_cols])
# submit["target"] = sub_pred
# submit.to_csv("mlp_optuna_logistic_fs_sub.csv", index=False)

In [ ]:
submit.shape

In [ ]:
submit.head